In [1]:
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/SMM4H 2024/SMM4H_2024_Task3_Training_1800.csv')
df_dev = pd.read_csv('/content/drive/MyDrive/SMM4H 2024/SMM4H_2024_Task3_Validation_600_release.csv')

In [2]:
df_train = df_train[:1800]
df_train = df_train.iloc[:, :4]
df_train['Classification'] = df_train['Classification'].astype(int)
df_train

,id,keyword,text,Classification
0,3u2w5k,run,"21/m. I want to experience young love, but I'v...",0
1,3xbury,outside,Having issues talking to a girl whom I enjoyed...,0
2,3y743u,"run, swim, climb",Need some advice for free social activities. I...,0
3,43bvs7,walk,I spoke to her today.. A few weeks ago I met ...,0
4,442ap2,outside,"How to get over Social Anxiety?. Hello, The ot...",0
...,...,...,...,...
1795,gqzye9,"pool, beach, pool",Social Anxiety: The Essentials. After looking...,3
1796,env299,"outside , outdoors",Eye contact. Being in public. I can't do it. ...,3
1797,e9bnr1s,Jogging,"If you look for the light, you will find it. I...",3
1798,qrmhbe,"walk, swimming, pool","I feel like my SA is ruining my life. Hi, I h...",3


In [3]:
!pip install datasets -U
!pip install transformers -U
!pip install accelerate -U
!pip install sentencepiece -U
!pip install tqdm -U

In [5]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
tokenizer = AutoTokenizer.from_pretrained("AmazonScience/qanlu")
model = AutoModelForQuestionAnswering.from_pretrained("AmazonScience/qanlu")
qa_pipeline = pipeline('question-answering', model=model, tokenizer=tokenizer)
qa_input = {
  'context': 'Yes. No. I want a cheap flight to Boston.',
  'question': 'What is the destination?'
}
answer = qa_pipeline(qa_input)

In [6]:
df_train

,id,keyword,text,Classification
0,3u2w5k,run,"21/m. I want to experience young love, but I'v...",0
1,3xbury,outside,Having issues talking to a girl whom I enjoyed...,0
2,3y743u,"run, swim, climb",Need some advice for free social activities. I...,0
3,43bvs7,walk,I spoke to her today.. A few weeks ago I met ...,0
4,442ap2,outside,"How to get over Social Anxiety?. Hello, The ot...",0
...,...,...,...,...
1795,gqzye9,"pool, beach, pool",Social Anxiety: The Essentials. After looking...,3
1796,env299,"outside , outdoors",Eye contact. Being in public. I can't do it. ...,3
1797,e9bnr1s,Jogging,"If you look for the light, you will find it. I...",3
1798,qrmhbe,"walk, swimming, pool","I feel like my SA is ruining my life. Hi, I h...",3


In [7]:
qa_pipeline = pipeline('question-answering', model=model, tokenizer=tokenizer)
qa_input_1 = {
  'context': "Yes. No. 21/m. I want to experience young love, but I've never had a relationship before. Am I going to miss out?. I have bad social anxiety and because of it not much social experience, but I think I can work through that within a year. Afterwards, I want to make a run at young love, but since I've never had any experience, I feel like I'll be building a bridge that will take a few years to build, and by that point I would have missed out. Given my situation, can I experience young love? Am I just going to have to miss out given my situation? tl;dr -- want to experience young love, but fear I don't have the experience to make it happen in time. Is this an unrealistic goal? ",
  'question': 'Did outdoor spaces and activities get mentioned ?'
}
qa_input_2 = {
  'context': "Yes. No. 21/m. I want to experience young love, but I've never had a relationship before. Am I going to miss out?. I have bad social anxiety and because of it not much social experience, but I think I can work through that within a year. Afterwards, I want to make a run at young love, but since I've never had any experience, I feel like I'll be building a bridge that will take a few years to build, and by that point I would have missed out. Given my situation, can I experience young love? Am I just going to have to miss out given my situation? tl;dr -- want to experience young love, but fear I don't have the experience to make it happen in time. Is this an unrealistic goal? ",
  'question': 'Did outdoor spaces and activities have any impact on the user ?'
}
qa_input_3 = {
  'context': "Yes. No. 21/m. I want to experience young love, but I've never had a relationship before. Am I going to miss out?. I have bad social anxiety and because of it not much social experience, but I think I can work through that within a year. Afterwards, I want to make a run at young love, but since I've never had any experience, I feel like I'll be building a bridge that will take a few years to build, and by that point I would have missed out. Given my situation, can I experience young love? Am I just going to have to miss out given my situation? tl;dr -- want to experience young love, but fear I don't have the experience to make it happen in time. Is this an unrealistic goal? ",
  'question': 'Did outdoor spaces and activities have a positive impact on the user ?'
}
answer_1 = qa_pipeline(qa_input_1)
print(answer_1['answer'])
answer_2 = qa_pipeline(qa_input_2)
print(answer_2['answer'])
answer_3 = qa_pipeline(qa_input_3)
print(answer_3['answer'])

No.
No.
No.


In [8]:
df_train['text'] = df_train['text'].str.replace('"', "'")
df_dev['text'] = df_dev['text'].str.replace('"', "'")

In [9]:
from tqdm import tqdm
qa_pipeline = pipeline('question-answering', model=model, tokenizer=tokenizer)
df_train['ans_1'] = ''
df_train['ans_2'] = ''
df_train['ans_3'] = ''
for index, row in tqdm(df_train.iterrows(), total=df_train.shape[0]):
    qa_input_1 = {   'context': "Yes. No. " + row['text'],      'question': 'Did outdoor spaces and activities get mentioned? '                       }
    qa_input_2 = {   'context': "Yes. No. " + row['text'],      'question': 'Did outdoor spaces and activities have any impact on the user?'          }
    qa_input_3 = {   'context': "Yes. No. " + row['text'],      'question': 'Did outdoor spaces and activities have a positive impact on the user?'   }
    answer_1 = qa_pipeline(qa_input_1)
    answer_2 = qa_pipeline(qa_input_2)
    answer_3 = qa_pipeline(qa_input_3)
    df_train.at[index, 'ans_1'] = answer_1['answer']
    df_train.at[index, 'ans_2'] = answer_2['answer']
    df_train.at[index, 'ans_3'] = answer_3['answer']

100%|██████████| 1800/1800 [22:28<00:00,  1.34it/s]


In [10]:
df_train

,id,keyword,text,Classification,ans_1,ans_2,ans_3
0,3u2w5k,run,"21/m. I want to experience young love, but I'v...",0,No.,No.,No.
1,3xbury,outside,Having issues talking to a girl whom I enjoyed...,0,No.,No.,No.
2,3y743u,"run, swim, climb",Need some advice for free social activities. I...,0,No.,No.,No.
3,43bvs7,walk,I spoke to her today.. A few weeks ago I met ...,0,No.,No.,No.
4,442ap2,outside,"How to get over Social Anxiety?. Hello, The ot...",0,No.,No.,No.
...,...,...,...,...,...,...,...
1795,gqzye9,"pool, beach, pool",Social Anxiety: The Essentials. After looking...,3,No.,No.,No.
1796,env299,"outside , outdoors",Eye contact. Being in public. I can't do it. ...,3,No.,No.,No.
1797,e9bnr1s,Jogging,"If you look for the light, you will find it. I...",3,No.,No.,No.
1798,qrmhbe,"walk, swimming, pool","I feel like my SA is ruining my life. Hi, I h...",3,No.,No.,No.


In [12]:
keyw_train = list(set([keyword.strip() for sublist in df_train['keyword'].dropna() for keyword in sublist.split(',')]))
keyw_train

['garden',
 'baseball',
 'walk',
 'jogging',
 'Go for a run',
 'nature',
 'cabin',
 'runner',
 'roller blade',
 'climb',
 'skating',
 'Fresh air',
 'streams',
 'camp',
 'runs',
 'outdoors',
 'backpacking',
 'running',
 'swim',
 'parks',
 'snowboard',
 'run',
 'lawn',
 'tree',
 'soccer',
 'Jogging',
 'Open Space',
 'tennis',
 'Outdoor',
 'bike',
 'mountains',
 'roller blades',
 'Go for a walk',
 'forest',
 'bench',
 'Garden',
 'Parks',
 'hill',
 'Hiking',
 'biking',
 'trees',
 'coast',
 'jogs',
 'horses',
 'lake',
 'mountain',
 'kayak',
 'beach',
 'hikes',
 'grass',
 'horse',
 'swimming',
 'outside',
 'bikes',
 'basketball',
 'pond',
 'sun',
 'waves',
 'parkour',
 'stream',
 'jog',
 'sea',
 'backyard',
 'cruise',
 'riding',
 'skate',
 'gardening',
 'boat',
 'waterfall',
 'park',
 'camps',
 'bicycle',
 'camping',
 'hiking',
 'hike',
 'golf',
 'pool']

In [ ]:
['garden','baseball','walk','jogging','Go for a run','nature','cabin','runner','roller blade','climb','skating','Fresh air','streams','camp','runs','outdoors','backpacking','running','swim','parks','snowboard','run','lawn','tree','soccer','Jogging','Open Space','tennis','Outdoor','bike','mountains','roller blades','Go for a walk','forest','bench','Garden','Parks','hill','Hiking','biking','trees','coast','jogs','horses','lake','mountain','kayak','beach','hikes','grass','horse','swimming','outside','bikes','basketball','pond','sun','waves','parkour','stream','jog','sea','backyard','cruise','riding','skate','gardening','boat','waterfall','park','camps','bicycle','camping','hiking','hike','golf','pool']